# 导包，设置根目录

In [3]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import random
import pickle
import multiprocessing
import re

from collections import Counter
from itertools import chain
import datetime

# 更改工作目录为当前项目根目录
import sys
import os
os.chdir(os.path.dirname(os.path.dirname(sys.path[0])))
print(os.getcwd())

C:\workfile\python\Log-diagnosis


# 读取数据

In [61]:
frequency_vector_df=pd.read_csv('pre_contest/v1_model/frequency_vector_df.csv',sep=',')
test_frequency_vector_df=pd.read_csv('pre_contest/v1_model/test_frequency_vector_df.csv',sep=',')
preliminary_submit_dataset=pd.read_csv('./pre_contest/dataset/preliminary_submit_dataset.txt',sep='\t')

In [63]:
test_frequency_vector_df

,asserted,power,system,memory,drive,initiated,deasserted,processor,boot,state,...,nic11,dimm031_status,nic13,dimm171_status,9120,dimm1b,memory_hot,00fc92f15f00,sn,fault_time
0,2,1,2,0,0,4,0,0,2,0,...,0,0,0,0,0,0,0,0,000d33b21436,2020-09-02 16:42:54
1,4,2,3,1,0,2,0,0,1,2,...,0,0,0,0,0,0,0,0,005c5a9218ba,2020-06-28 19:05:16
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0079283bde6e,2020-04-26 21:32:44
3,5,2,3,1,0,2,0,0,1,2,...,0,0,0,0,0,0,0,0,007bdf23b62f,2020-06-16 18:40:39
4,6,0,3,2,0,1,0,0,3,0,...,0,0,0,0,0,0,0,0,00a577a8e54f,2020-04-07 07:16:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ffbf46b4af21,2019-12-28 20:10:01
3007,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ffc229b6cd9a,2020-06-27 02:39:08
3008,2,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,ffd44698a52b,2020-01-21 15:46:56
3009,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fff73a9e5bd5,2020-03-01 22:43:43


# 训练XGB模型

In [62]:
colnames=frequency_vector_df.columns
feature=np.array(frequency_vector_df[colnames[:-3]])
label=np.array(frequency_vector_df['label'])
test_feature=np.array(test_frequency_vector_df[colnames[:-3]])
train_data=xgb.DMatrix(feature,label=label)
test_data=xgb.DMatrix(test_feature)

In [52]:
# xgb模型参数
params = {
    'booster':'gbtree',
    'objective':'multi:softmax',   # 多分类问题
    'num_class':4,  # 类别数，与multi softmax并用
    'gamma':0.1,    # 用于控制是否后剪枝的参数，越大越保守，一般0.1 0.2的样子
    'max_depth':6,  # 构建树的深度，越大越容易过拟合
    'lambda':2,  # 控制模型复杂度的权重值的L2 正则化项参数，参数越大，模型越不容易过拟合
    'subsample':1, # 随机采样训练样本
    'colsample_bytree':1,# 这个参数默认为1，是每个叶子里面h的和至少是多少
    # 对于正负样本不均衡时的0-1分类而言，假设h在0.01附近，min_child_weight为1
    #意味着叶子节点中最少需要包含100个样本。这个参数非常影响结果，
    # 控制叶子节点中二阶导的和的最小值，该参数值越小，越容易过拟合
    'silent':0,  # 设置成1 则没有运行信息输入，最好是设置成0
    'eta':0.3,  # 如同学习率
    'seed':1000,
    'nthread':12,  #CPU线程数
    #'eval_metric':'auc'
}
xgb_model_V1=xgb.train(params,train_data,num_boost_round=500)

[00:09:48] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[00:09:48] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [57]:
# file = open('./pre_contest/v1_model/xgb_model_V1.model','wb')
# pickle.dump(xgb_model_V1, file)

In [65]:
preliminary_submit_dataset['prediction']=xgb_model_V1.predict(test_data)

In [66]:
preliminary_submit_dataset_a=preliminary_submit_dataset[['sn','fault_time','prediction']]
preliminary_submit_dataset_a['label']=preliminary_submit_dataset_a['prediction'].apply(lambda x : int(x))
preliminary_submit_dataset_a=preliminary_submit_dataset_a[['sn','fault_time','label']]
# preliminary_submit_dataset_a.to_csv('pre_contest/v1_model/preliminary_submit_dataset_a_V1.csv',sep=',',index=None)

In [67]:
preliminary_submit_dataset_a

,sn,fault_time,label
0,000d33b21436,2020-09-02 16:42:54,3
1,005c5a9218ba,2020-06-28 19:05:16,2
2,0079283bde6e,2020-04-26 21:32:44,3
3,007bdf23b62f,2020-06-16 18:40:39,2
4,00a577a8e54f,2020-04-07 07:16:55,2
...,...,...,...
3006,ffbf46b4af21,2019-12-28 20:10:01,2
3007,ffc229b6cd9a,2020-06-27 02:39:08,2
3008,ffd44698a52b,2020-01-21 15:46:56,2
3009,fff73a9e5bd5,2020-03-01 22:43:43,2
